## 誘導部分グラフ同型問題を扱うjupyter

ライブラリのインポート

In [1]:
import numpy as np
import datetime
import IsingLib

パラメータの設定

In [28]:
param = {}

#graph A
param['num_a'] = 50
param['density_a'] = 0.5

#graph B
param['num_b'] = 10
#param['density_b'] = 0.5

グラフの生成

In [29]:
def generate_graph(n, density):
    a_matrix = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(i+1, n):
            if np.random.rand() < density:
                a_matrix[i][j] = 1
                a_matrix[j][i] = 1
    return a_matrix

def extract_graph(matrix_src, n):
    a_matrix = np.zeros((n, n), dtype=int)
    index = np.random.choice(range(matrix_src.shape[0]), n, replace=False)
    for i in range(n):
        for j in range(i+1, n):
            a_matrix[i][j] = matrix_src[index[i]][index[j]]
            a_matrix[j][i] = matrix_src[index[i]][index[j]]
    return a_matrix

In [30]:
param['matrix_a'] = generate_graph(param['num_a'], param['density_a'])
param['matrix_b'] = extract_graph(param['matrix_a'], param['num_b'])

グラフの表示

In [31]:
print('グラフA')
print(param['matrix_a'])
print('\nグラフB')
print(param['matrix_b'])

グラフA
[[0 1 0 ... 1 0 1]
 [1 0 1 ... 0 0 1]
 [0 1 0 ... 1 0 1]
 ...
 [1 0 1 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [1 1 1 ... 1 1 0]]

グラフB
[[0 0 0 1 0 0 1 1 0 0]
 [0 0 1 0 1 1 0 0 0 0]
 [0 1 0 1 0 1 0 1 0 1]
 [1 0 1 0 1 1 1 1 0 0]
 [0 1 0 1 0 1 1 1 1 1]
 [0 1 1 1 1 0 1 1 1 1]
 [1 0 0 1 1 1 0 1 0 1]
 [1 0 1 1 1 1 1 0 0 0]
 [0 0 0 0 1 1 0 0 0 0]
 [0 0 1 0 1 1 1 0 0 0]]


## QUBO係数行列の生成

パラメータの生成

In [32]:
param['alpha'] = 1
param['beta'] = 1
param['gamma'] = 3
param['delta'] = 3

param['num_qubo'] = param['num_a'] * param['num_b']
param['num_column'] = param['num_a']
param['num_row'] = param['num_b']
# 置換行列は横長

In [33]:
qubo = {}
qubo['const'] = np.zeros(1, dtype=int)
qubo['bias'] = np.zeros(param['num_qubo'], dtype=int)
qubo['weight'] = np.zeros((param['num_qubo'], param['num_qubo']), dtype=int)

def get_index(tuple_p, param):
    i, j = tuple_p
    return param['num_a']*i+j

def get_tuple(index_p, param):
    i, j = index_p//param['num_a'], index_p%param['num_a']
    return i, j

- 誘導部分グラフ頂点割当重複禁止制約

In [34]:
qubo['const'] += param['alpha']*param['num_row']
qubo['bias'] += -param['alpha']
for i in range(param['num_row']):
    for u in range(param['num_column']-1):
        for v in range(u+1, param['num_column']):
            qubo['weight'][get_index((i,u), param)][get_index((i,v), param)] += 2*param['alpha']

- グラフ頂点割当制約

In [35]:
qubo['const'] += param['beta']*param['num_column']//4 # assume param['num_column'] % 4 == 0
for u in range(param['num_column']):
    for i in range(param['num_row']-1):
        for j in range(i+1, param['num_row']):
            qubo['weight'][get_index((i, u), param)][get_index((j, u), param)] += 2*param['beta']

- 接続増加禁止制約

In [36]:
for i in range(param['num_b']-1):
    for j in range(i+1, param['num_b']):
        # 接続しているなら制約なし
        if param['matrix_b'][i][j] == 1:
            continue
        for u in range(param['num_a']-1):
            for v in range(u+1, param['num_a']):
                # 接続していないなら制約なし
                if param['matrix_a'][u][v] == 0:
                    continue
                # 無向グラフなので割り当てが2パターンある
                qubo['weight'][get_index((i, u), param)][get_index((j, v), param)] += param['gamma']
                qubo['weight'][get_index((i, v), param)][get_index((j, u), param)] += param['gamma']

- 接続減少禁止制約

In [37]:
for i in range(param['num_b']-1):
    for j in range(i+1, param['num_b']):
        # 接続していないなら制約なし
        if param['matrix_b'][i][j] == 0:
            continue
        for u in range(param['num_a']-1):
            for v in range(u+1, param['num_a']):
                # 接続しているなら制約なし
                if param['matrix_a'][u][v] == 1:
                    continue
                # 無向グラフなので割り当てが2パターンある
                qubo['weight'][get_index((i, u), param)][get_index((j, v), param)] += param['delta']
                qubo['weight'][get_index((i, v), param)][get_index((j, u), param)] += param['delta']

QUBO行列を表示

In [38]:
#np.set_printoptions(threshold=1000)
#print('QUBO行列')
print(qubo)
mat = qubo['weight'] + np.diag(qubo['bias'])
print(mat)

{'const': array([22]), 'bias': array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1

npzファイルに保存

In [39]:
model = IsingLib.Model('qubo')

In [40]:
model.formulate(H=mat)
model.compile()
sta = model.solve()
const = list(sta.values())[0]
print(const)

-8.0


In [41]:
if const+qubo['const'][0] == param['beta']*param['num_a']//4:
    print('\n基底解が得られた!')
else:
    print('\n基底解が得られていない')


基底解が得られていない
